In [1]:
import numpy as np
import gym
from numpy.random import choice
import random
from tensorbuilder.api import *
import tensorflow as tf
from tensorflow.contrib import layers
import numpy as np

env = gym.make("FrozenLake-v0")


hdf5 not supported (please install/reinstall h5py)


[2017-01-26 15:22:48,330] Making new env: FrozenLake-v0


In [5]:
n_actions = env.action_space.n
n_states = env.observation_space.n
learning_rate = 0.85
y = 0.9
k = 20000.0
model_name = "policy-gradient.model"
model_path = "/models/" + model_name

graph = tf.Graph()
gsess = tf.InteractiveSession(graph=graph)
with graph.as_default():
    with tf.device("cpu:0"):
        s = tf.placeholder(tf.int32, ())
        step = tf.placeholder(tf.int32, ())
        a = tf.placeholder(tf.int32, ())
        r = tf.placeholder(tf.float32, ())
        

        ops = dict(trainable=True, weights_initializer=tf.random_uniform_initializer(minval=0, maxval=0.01), biases_initializer=None) #tf.random_uniform_initializer(minval=0, maxval=0.01))


        Ps = Pipe(
            s,
            T.one_hot(n_states).expand_dims(0)
#             .relu_layer(32, scope='relu_layer', **ops),
            .softmax_layer(n_actions, scope='softmax_layer', **ops),
            T[0]
        )
        Psa = Ps[a]
        
        ws = tf.trainable_variables()
        gradients = tf.gradients(tf.log(Psa), ws)
        dws = [ tf.placeholder(dw.dtype, dw.get_shape()) for dw in gradients ]
        

        update = [ tf.assign_add(w, ws) for w, ws in zip(ws, dws) ]

        writer = tf.summary.FileWriter('/logs/' +  model_name)
        saver = tf.train.Saver()

In [6]:
def next_action(actions, get_max=False):
    n = actions.shape[0]
    return choice(n, p=actions) if not get_max else np.argmax(actions)
    
def discount(rewards, y):
    r_accum = 0.0
    for i in reversed(range(len(rewards))):
        r_accum = rewards[i] = rewards[i] + y * r_accum
        
    return rewards
            
            
with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.global_variables_initializer())
#     saver.restore(sess, model_path)
    
    r_total = 0
    
    for t in range(200000):
        
        lr = 0.5 * k / (k + t)
        
        #reset lists    
        s_list = []
        r_list = []
        a_list = []
                
        
        _s = env.reset()
        
        
        while True:
            #calculate Ps
            _qs = sess.run(Ps, feed_dict={s: _s})
            
            #select action
            _a = next_action(_qs)
            
            #take step
            _s1, _r, done, info = env.step(_a)
            
            r_total += _r
            
            #store values
            s_list.append(_s)
            r_list.append(_r)
            a_list.append(_a)

            #update state
            _s = _s1

            if done:
                break
        
        gains = discount(r_list, y)
        _dws = None
        
        for _s, gain, _a in zip(s_list, gains, a_list):
            #get policy gradient
            dp_dws = sess.run(gradients, {s: _s, a: _a})
            
            #step update
            dw_updates = map(lambda dp_dw: gain * dp_dw, dp_dws)
            
            #get weights gradient
            if _dws is None:
                _dws = dw_updates
            else:
                _dws = [ _dw + dw_update for _dw, dw_update in zip(_dws, dw_updates) ]
                
        #apply gradient
        _dws = [ lr * _dw for _dw in _dws ]
        sess.run(update, feed_dict={dw: _dw for dw, _dw in zip(dws, _dws)})
            
        

        if t % 500 == 0:
            print int(r_total), "of", 500
#             print(t_ward)
            r_total = 0
            saver.save(sess, model_name)


0 of 500
13 of 500
14 of 500
27 of 500
88 of 500
123 of 500
162 of 500
270 of 500


KeyboardInterrupt: 

In [ ]:
tf.gather(Ps, a)

In [8]:
_s = env.reset()
done = False
with tf.Session(graph=graph) as sess:
    saver.restore(sess, model_path)
    
    for i in range(100):
        _qs = sess.run(Ps, feed_dict={s: _s})
        _a = next_action(_qs, 0)
        _s, _r, done, info = env.step(_a)
        env.render()
        print("")

        if done:
            print(_r)
            break

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Right)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Up)

SFFF
FHFH
FFFH
HFFG
  (Down)

SFFF
FHFH
FFFH
HFFG
  (Left)

SFFF
FHFH
FFFH
HFFG
  (Right)

1.0
